In [ ]:
%matplotlib inline
%config InlineBackend.figure_format = "retina"
from matplotlib import rcParams
rcParams["savefig.dpi"] = 100
rcParams["figure.dpi"] = 100

import numpy as np
import matplotlib.pyplot as plt

In [ ]:
def step1(dx, dy):
    n = len(dx)
    np1 = n + 1
    
    a = np.empty(np1)
    a[0] = 3 * dy[0] / dx[0]
    a[1:-1] = 3 * dy[1:] / dx[1:] - 3 * dy[:-1] / dx[:-1]
    a[-1] = -3 * dy[-1] / dx[-1]
    
    return a

def step1_rev(dx, dy, a, ba):
    bdx = np.zeros_like(dx)
    bdy = np.zeros_like(dy)
    
    # a[0] = 3 * dy[0] / dx[0]
    bdy[0] += 3 * ba[0] / dx[0]
    bdx[0] += -a[0] * ba[0] / dx[0]

    # a[1:-1] = 3 * dy[1:] / dx[1:] - 3 * dy[:-1] / dx[:-1]
    bdy[1:] += 3 * ba[1:-1] / dx[1:]
    bdy[:-1] += -3 * ba[1:-1] / dx[:-1]
    bdx[1:] += -3 * dy[1:] * ba[1:-1] / dx[1:]**2
    bdx[:-1] += 3 * dy[:-1] * ba[1:-1] / dx[:-1]**2

    # a[-1] = -3 * dy[-1] / dx[-1]
    bdy[-1] += -3 * ba[-1] / dx[-1]
    bdx[-1] += -a[-1] * ba[-1] / dx[-1]
    
    return bdx, bdy

def step2(dx, a):
    n = len(dx)
    np1 = n + 1

    l = np.empty(np1)
    u = np.empty(n)
    z = np.empty(np1)
    l[0] = 2*dx[0]
    u[0] = 0.5
    z[0] = a[0] / l[0]
    for i in range(1, n):
        l[i] = 2*dx[i] + dx[i-1] * (2 - u[i-1])
        u[i] = dx[i] / l[i]
        z[i] = (a[i] - dx[i-1] * z[i-1]) / l[i]
    l[-1] = dx[-1] * (2 - u[-1])
    z[-1] = (a[-1] - dx[-1] * z[-2]) / l[-1]
    
    return u, l, z

def step2_rev(dx, a, u, l, z, bu, bl, bz):
    n = len(u)
    
    bu = np.array(bu)
    bl = np.array(bl)
    bz = np.array(bz)

    ba = np.zeros_like(a)
    bdx = np.zeros_like(dx)

    # z[-1] = (a[-1] - dx[-1] * z[-2]) / l[-1]
    ba[-1] += bz[-1] / l[-1]
    bdx[-1] += -z[-2] * bz[-1] / l[-1]
    bz[-2] += -dx[-1] * bz[-1] / l[-1]
    bl[-1] += -z[-1] * bz[-1] / l[-1]
    
    # l[-1] = dx[-1] * (2 - u[-1])
    bdx[-1] += (2 - u[-1]) * bl[-1]
    bu[-1] += -dx[-1] * bl[-1]

    # for i in range(1, n):
    for i in range(n-1, 0, -1):
        # z[i] = (a[i] - dx[i-1] * z[i-1]) / l[i]
        ba[i] += bz[i] / l[i]
        bl[i] += -z[i]*bz[i]/l[i]
        bdx[i-1] += -z[i-1] * bz[i] / l[i]
        bz[i-1] += -bz[i] * dx[i-1] / l[i]
        
        # u[i] = dx[i] / l[i]
        bdx[i] += bu[i] / l[i]
        bl[i] += -bu[i]*u[i]/l[i]

        # l[i] = 2*dx[i] + dx[i-1] * (2 - u[i-1])
        bdx[i] += 2*bl[i]
        bdx[i-1] += (2-u[i-1])*bl[i]
        bu[i-1] += -dx[i-1] * bl[i]
        
    # z[0] = a[0] / l[0]
    ba[0] += bz[0] / l[0]
    bl[0] += -z[0] * bz[0] / l[0]

    # l[0] = 2*dx[0]
    bdx[0] += 2*bl[0]
    return bdx, ba

def step3(z, u):
    n = len(u)
    c = np.empty_like(z)
    c[-1] = z[-1]
    for j in range(n-1, -1, -1):
        c[j] = z[j] - u[j] * c[j+1]
    return c

def step3_rev(z, u, c, bc):
    n = len(u)
    bc = np.array(bc)
    bu = np.zeros_like(u)
    bz = np.zeros_like(z)
    # for j in range(n-1, -1, -1):
    for j in range(n):
        # c[j] = z[j] - u[j] * c[j+1]
        bz[j] += bc[j]
        bc[j+1] += -bc[j] * u[j]
        bu[j] += -c[j+1] * bc[j]
    # c[-1] = z[-1]
    bz[-1] += bc[-1]
    return bz, bu

def step4(dx, dy, c):
    b = dy / dx - dx * (c[1:] + 2*c[:-1]) / 3
    d = (c[1:] - c[:-1]) / (3*dx)
    return b, d

def step4_rev(dx, dy, c, b, d, bb, bd):
    bc = np.zeros_like(c)
    
    # d = (c[1:] - c[:-1]) / (3*dx)
    bdx = -d * bd / dx
    bc[1:] += bd / (3*dx)
    bc[:-1] += -bd / (3*dx)
    
    # b = dy / dx - dx * (c[1:] + 2*c[:-1]) / 3
    bdy = bb / dx
    bdx += -(dy/dx**2 + (c[1:]+2*c[:-1])/3) * bb
    bc[1:] += -dx * bb / 3
    bc[:-1] += -2 * dx * bb / 3

    return bdx, bdy, bc

def compute_polys(dx, dy):
    n = len(dx)
    np1 = n + 1
    
    # Step 1
    a = step1(dx, dy)
    
    # Step 2
    u, l, z = step2(dx, a)
    
    # Step 3
    c = step3(z, u)    
    
    # Step 4
    b, d = step4(dx, dy, c)
    
    return (np.vstack((
        np.concatenate(([0.0], b, [0.0])),
        np.concatenate(([0.0], c[:-1], [0.0])),
        np.concatenate(([0.0], d, [0.0]))
    )).T, a, z, u, l)

def compute_polys_rev(dx, dy, P, a, z, u, l, bP):
    n = len(dx)
    np1 = n + 1

    b = P[1:-1, 0]
    c = P[1:, 1]
    d = P[1:-1, 2]
    
    bb = np.array(bP[1:-1, 0])
    bc = np.array(bP[1:, 1])
    bd = np.array(bP[1:-1, 2])
    bc[-1] = 0.0
    
    # Step 4
    bdx, bdy, bc0 = step4_rev(dx, dy, c, b, d, bb, bd)
    bc += bc0
    
    # Step 3
    bz, bu = step3_rev(z, u, c, bc)
    
    # Step 2
    bl = np.zeros_like(l)
    bdx0, ba = step2_rev(dx, a, u, l, z, bu, bl, bz)
    bdx += bdx0
    
    # Step 1
    bdx0, bdy0 = step1_rev(dx, dy, a, ba)
    bdx += bdx0
    bdy += bdy0
    
    return bdx, bdy

In [ ]:
def check_grad(value, grad, f, args=None, eps=1e-8, ind=None, factor=None):
    if args is None:
        args = (value,)
    if factor is None:
        factor = 1.0
    for i in range(len(value)):
        value[i] += eps
        r = f(*args)
        if ind is None:
            vp = np.sum(factor*r)
        else:
            vp = np.sum(factor*r[ind])

        value[i] -= 2*eps
        r = f(*args)
        if ind is None:
            vm = np.sum(factor*r)
        else:
            vm = np.sum(factor*r[ind])
        value[i] += eps

        est = 0.5 * (vp - vm) / eps
        print(est, grad[i], est - grad[i])

In [ ]:
n = 5
dx = np.random.rand(n)
dy = np.random.randn(n)
c = np.random.randn(n+1)
b, d = step4(dx, dy, c)

bb = np.random.randn(len(b))
bd = np.zeros_like(d)
bdx, bdy, bc = step4_rev(dx, dy, c, b, d, bb, bd)

print("b, dx:")
check_grad(dx, bdx, step4, args=(dx, dy, c), ind=0, factor=bb)
print("b, dy:")
check_grad(dy, bdy, step4, args=(dx, dy, c), ind=0, factor=bb)
print("b, c:")
check_grad(c, bc, step4, args=(dx, dy, c), ind=0, factor=bb)

bb = np.zeros_like(b)
bd = np.random.randn(len(d))
bdx, bdy, bc = step4_rev(dx, dy, c, b, d, bb, bd)

print("d, dx:")
check_grad(dx, bdx, step4, args=(dx, dy, c), ind=1, factor=bd)
print("d, dy:")
check_grad(dy, bdy, step4, args=(dx, dy, c), ind=1, factor=bd)
print("d, c:")
check_grad(c, bc, step4, args=(dx, dy, c), ind=1, factor=bd)


In [ ]:
n = 5
u = np.random.randn(n)
z = np.random.randn(n+1)
c = step3(z, u)
bc = np.random.randn(len(c))
bz, bu = step3_rev(z, u, c, bc)

print("u:")
check_grad(u, bu, step3, args=(z, u), factor=bc)
print("z:")
check_grad(z, bz, step3, args=(z, u), factor=bc)

In [ ]:
n = 5
dx = np.random.rand(n)
a = np.random.randn(n+1)
u, l, z = step2(dx, a)

bu = np.random.randn(len(u))
bl = np.zeros_like(l)
bz = np.zeros_like(z)
bdx, ba = step2_rev(dx, a, u, l, z, bu, bl, bz)

print("u, dx:")
check_grad(dx, bdx, step2, args=(dx, a), ind=0, factor=bu)
print("u, a:")
check_grad(a, ba, step2, args=(dx, a), ind=0, factor=bu)

bu = np.zeros_like(u)
bl = np.random.randn(len(l))
bz = np.zeros_like(z)
bdx, ba = step2_rev(dx, a, u, l, z, bu, bl, bz)

print("l, dx:")
check_grad(dx, bdx, step2, args=(dx, a), ind=1, factor=bl)
print("l, a:")
check_grad(a, ba, step2, args=(dx, a), ind=1, factor=bl)

bu = np.zeros_like(u)
bl = np.zeros_like(l)
bz = np.random.randn(len(z))
bdx, ba = step2_rev(dx, a, u, l, z, bu, bl, bz)

print("z, dx:")
check_grad(dx, bdx, step2, args=(dx, a), ind=2, factor=bz)
print("z, a:")
check_grad(a, ba, step2, args=(dx, a), ind=2, factor=bz)

In [ ]:
n = 5
dx = np.random.rand(n)
dy = np.random.randn(n)
a = step1(dx, dy)
ba = np.random.randn(len(a))
bdx, bdy = step1_rev(dx, dy, a, ba)

print("dx:")
check_grad(dx, bdx, step1, args=(dx, dy), factor=ba)
print("dy:")
check_grad(dy, bdy, step1, args=(dx, dy), factor=ba)

In [ ]:
bc

In [ ]:
np.random.seed(42)
x = np.sort(np.random.uniform(1, 9, 8))
# x = np.linspace(1, 9, 10)
y = np.sin(x)
dx = np.diff(x)
dy = np.diff(y)

P, a, z, u, l = compute_polys(dx, dy)
bP = np.zeros_like(P)
# inds = ([-3], [2])
inds = tuple(a.flatten() for a in np.indices(bP.shape))
bP[inds] = 1.0
# print(bP)
bx, by = compute_polys_rev(dx, dy, P, a, z, u, l, bP)
# print(bx)
# print(by)

In [ ]:
value = dx
grad = bx

eps = 1e-5

for i in range(len(value)):
    value[i] += eps
    r = compute_polys(dx, dy)
    vp = np.sum(r[0][inds])

    value[i] -= 2*eps
    r = compute_polys(dx, dy)
    vm = np.sum(r[0][inds])
    value[i] += eps

    est = 0.5 * (vp - vm) / eps
    print(est, grad[i], est - grad[i])

In [ ]:
t = np.linspace(0, 10, 500)
m = np.searchsorted(x, t)

xp = np.concatenate((x[:1], x, x[-1:]))
yp = np.concatenate((y[:1], y, y[-1:]))

poly = P[m]
dd = t - xp[m]
value = yp[m] + poly[:, 0] * dd + poly[:, 1] * dd**2 + poly[:, 2] * dd**3

plt.plot(t, np.sin(t))
plt.plot(t, value)
plt.plot(x, y, ".")

In [ ]:
def get_system(dx, dy):
    A = np.diag(np.concatenate((
        2*dx[:1], 2*(dx[1:]+dx[:-1]), 2*dx[-1:]
    )))
    A += np.diag(dx, k=1)
    A += np.diag(dx, k=-1)
    
    Y = np.concatenate((
        3 * dy[:1]/dx[:1],
        3 * (dy[1:]/dx[1:] - dy[:-1]/dx[1:]),
        -3 * dy[:1]/dx[:1],
    ))
    
    return A, Y

In [ ]:
A, Y = get_system(dx, dy)

In [ ]:
c = np.linalg.solve(A, Y)
c

In [ ]:
bc = np.random.randn(len(c))
Ax = np.linalg.solve(A, bc)
bA = -Ax[:, None] * c[None, :]

In [ ]:
bA

In [ ]:
print(np.allclose(np.diag(bA), -Ax*c))
print(np.allclose(np.diag(bA, k=1), -Ax[:-1]*c[1:]))
print(np.allclose(np.diag(bA, k=-1), -Ax[1:]*c[:-1]))

In [ ]:
eps = 1e-5

A[1, 0] += eps
r = np.linalg.solve(A, Y)
vp = np.sum(r * bc)

A[1, 0] -= 2*eps
r = np.linalg.solve(A, Y)
vm = np.sum(r * bc)
A[1, 0] += eps

est = 0.5 * (vp - vm) / eps
print(est, bA[1, 0], est - bA[1, 0])